In [1]:
import pandas as pd
import numpy as np
from functions import *
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegressionCV
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from sklearn.mixture import DPGMM

remove_fight_island = False

In [2]:
#Turn off warnings

import warnings
warnings.filterwarnings("ignore")

In [3]:
#Load models
#REMINDER: We are going to need to use 'eval' to get the models usable
with open('../data/models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
print(len(models))


###SELECT MODEL TO OPTIMIZE
model_num = 19

5


In [4]:
models

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_5',
  'model_6',
  'model_7',
  'model_8',
  'model_9',
  'model_10',
  'model_11',
  'model_12',
  'model_13',
  'model_14',
  'model_15',
  'model_16',
  'model_17',
  'model_18',
  'model_19'],
 ["LogisticRegression(max_iter=50000, random_state=75, solver='sag', tol=0.00012)",
  "DecisionTreeClassifier(max_depth=10, max_leaf_nodes=336,\n                       min_samples_leaf=0.0099, random_state=75,\n                       splitter='random')",
  "RandomForestClassifier(criterion='entropy', max_depth=83, max_leaf_nodes=20,\n                       min_samples_leaf=0.009999000000000001, n_estimators=140,\n                       n_jobs=-1, random_state=75)",
  'GradientBoostingClassifier(max_depth=95, min_samples_leaf=0.0015,\n                           n_estimators=195, random_state=75)',
  'GaussianNB()',
  "ExtraTreeClassifier(criterion='entropy', max_depth=83, max_leaf_nodes=20,\n                    min_sampl

In [5]:
df = pd.read_csv("../data/kaggle_data/ufc-master.csv")

In [6]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

In [7]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [8]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [9]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[250:]
odds_train = odds_df[250:]
label_train = label_df[250:]

df_test = df[:250]
odds_test = odds_df[:250]
label_test = label_df[:250]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

250
250
250
4521
4521
4521


In [10]:
if remove_fight_island:
    ##Let's remove the Fight island contests and see how that affects score
    df_test_no_fight_island = df_test[(df_test['location'] != 'Abu Dhabi, Abu Dhabi, United Arab Emirates')]
    df_train_no_fight_island = df_train[(df_train['location'] != 'Abu Dhabi, Abu Dhabi, United Arab Emirates')]
    df_test = df_test_no_fight_island
    df_train = df_train_no_fight_island



print(len(df_test))
print(len(df_train))


250
4521


In [11]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_pass_bout,R_rev_bout,B_rev_bout,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds,label
250,Marlon Moraes,Rob Font,-170,150,58.823529,150.000000,2020-12-19,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,235.0,250.0,600.0,1200.0,275.0,350.0,1
251,Marcin Tybura,Greg Hardy,-110,-110,90.909091,90.909091,2020-12-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,225.0,550.0,475.0,1800.0,650.0,145.0,0
252,Anthony Pettis,Alex Morono,-210,175,47.619048,175.000000,2020-12-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,150.0,325.0,600.0,1000.0,275.0,850.0,0
253,Sijara Eubanks,Pannie Kianzad,-140,120,71.428571,120.000000,2020-12-19,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,150.0,155.0,500.0,1400.0,650.0,1200.0,1
254,Deron Winn,Antonio Arroyo,160,-185,160.000000,54.054054,2020-12-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,225.0,300.0,1100.0,275.0,675.0,300.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4766,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1
4767,John Howard,Daniel Roberts,-210,175,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0
4768,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0
4769,Mike Pierce,Julio Paulino,-420,335,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_pass_bout,R_rev_bout,B_rev_bout,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds,label
0,Ciryl Gane,Alexander Volkov,-155,125,64.516129,125.000000,2021-06-26,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,188.0,300.0,700.0,375.0,275.0,1600.0,0
1,Tanner Boser,Ovince Saint Preux,-195,153,51.282051,153.000000,2021-06-26,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,175.0,250.0,1800.0,350.0,700.0,500.0,0
2,Raoni Barcelos,Timur Valiev,-230,180,43.478261,180.000000,2021-06-26,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,125.0,375.0,600.0,325.0,800.0,1250.0,1
3,Tim Means,Nicolas Dalby,-143,113,69.930070,113.000000,2021-06-26,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,165.0,400.0,800.0,225.0,600.0,1000.0,0
4,Renato Moicano,Jai Herbert,-250,195,40.000000,195.000000,2021-06-26,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,240.0,550.0,175.0,550.0,325.0,1500.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Sarah Moras,Vanessa Melo,-230,180,43.478261,180.000000,2021-01-16,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,-110.0,250.0,600.0,1200.0,650.0,1200.0,1
246,Jacob Kilburn,Austin Lingo,180,-230,180.000000,43.478261,2021-01-16,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,450.0,250.0,750.0,325.0,650.0,225.0,1
247,Stephen Thompson,Geoff Neal,-103,-117,97.087379,85.470085,2020-12-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,275.0,325.0,2000.0,1400.0,250.0,200.0,0
248,Jose Aldo,Marlon Vera,-140,120,71.428571,120.000000,2020-12-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,150.0,300.0,800.0,800.0,500.0,400.0,0


In [12]:
#Set a value for the nulls in the ranks

weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)


In [13]:
df_test
#df_test.to_csv('test.csv')

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_pass_bout,R_rev_bout,B_rev_bout,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds,label
0,Ciryl Gane,Alexander Volkov,-155,125,64.516129,125.000000,2021-06-26,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,188.0,300.0,700.0,375.0,275.0,1600.0,0
1,Tanner Boser,Ovince Saint Preux,-195,153,51.282051,153.000000,2021-06-26,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,175.0,250.0,1800.0,350.0,700.0,500.0,0
2,Raoni Barcelos,Timur Valiev,-230,180,43.478261,180.000000,2021-06-26,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,125.0,375.0,600.0,325.0,800.0,1250.0,1
3,Tim Means,Nicolas Dalby,-143,113,69.930070,113.000000,2021-06-26,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,165.0,400.0,800.0,225.0,600.0,1000.0,0
4,Renato Moicano,Jai Herbert,-250,195,40.000000,195.000000,2021-06-26,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,240.0,550.0,175.0,550.0,325.0,1500.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Sarah Moras,Vanessa Melo,-230,180,43.478261,180.000000,2021-01-16,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,-110.0,250.0,600.0,1200.0,650.0,1200.0,1
246,Jacob Kilburn,Austin Lingo,180,-230,180.000000,43.478261,2021-01-16,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,450.0,250.0,750.0,325.0,650.0,225.0,1
247,Stephen Thompson,Geoff Neal,-103,-117,97.087379,85.470085,2020-12-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,275.0,325.0,2000.0,1400.0,250.0,200.0,0
248,Jose Aldo,Marlon Vera,-140,120,71.428571,120.000000,2020-12-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,150.0,300.0,800.0,800.0,500.0,400.0,0


In [14]:
#1. Set features
#2. Set Hyperparameters
#3. Set EV
#4. Remove Features

In [15]:
models[1][model_num]

'SVC(probability=True, random_state=75)'

In [16]:
test_model_name = models[0][model_num]
test_model = eval(models[1][model_num])
test_model_features = eval(models[2][model_num])
test_model_ev = eval(models[3][model_num])

In [17]:
old_test_model = test_model
old_test_model_features = test_model_features
old_test_model_ev = test_model_ev

In [18]:
print(df_test.columns.tolist()) #Keeps us from truncating

['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev', 'date', 'location', 'country', 'Winner', 'title_bout', 'weight_class', 'gender', 'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak', 'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT', 'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses', 'B_total_rounds_fought', 'B_total_title_bouts', 'B_win_by_Decision_Majority', 'B_win_by_Decision_Split', 'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission', 'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms', 'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak', 'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed', 'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct', 'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought', 'R_total_title_bouts', 'R_win_by_Decision_Majority', 'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission',

In [19]:
#1. set features
my_pos_features = ['R_odds', 'B_odds', 'R_ev', 'B_ev',
       'location', 'country', 'title_bout', 'weight_class', 'gender',
       'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak',
       'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT',
       'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses',
       'B_total_rounds_fought', 'B_total_title_bouts',
       'B_win_by_Decision_Majority', 'B_win_by_Decision_Split',
       'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission',
       'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms',
       'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak',
       'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed',
       'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct',
       'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought',
       'R_total_title_bouts', 'R_win_by_Decision_Majority',
       'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous',
       'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Stance', 'R_Height_cms',
       'R_Reach_cms', 'R_Weight_lbs', 'R_age', 'B_age', 'lose_streak_dif',
       'win_streak_dif', 'longest_win_streak_dif', 'win_dif', 'loss_dif',
       'total_round_dif', 'total_title_bout_dif', 'ko_dif', 'sub_dif',
       'height_dif', 'reach_dif', 'age_dif', 'sig_str_dif', 'avg_sub_att_dif',
       'avg_td_dif', 'empty_arena', 'B_match_weightclass_rank', 'R_match_weightclass_rank', 
        "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank",
        "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 
        'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 
        'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", 
        "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 
        'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 
        'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 
        'B_Pound-for-Pound_rank', 'better_rank']

In [20]:
print(test_model_name)
print(test_model)
print(test_model_features)
print(test_model_ev)



model_19
SVC(probability=True, random_state=75)
['B_total_title_bouts', 'R_win_by_TKO_Doctor_Stoppage', 'lose_streak_dif', 'R_draw', 'B_draw', 'B_Welterweight_rank', 'loss_dif', 'R_age', 'B_Stance', 'B_ev', 'R_Weight_lbs', 'B_age', 'reach_dif', 'R_avg_TD_landed', 'B_avg_TD_pct']
0


In [21]:
def save_model():
    score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)
    models[0][model_num] =  test_model_name 
    models[1][model_num] = test_model
    models[2][model_num] = test_model_features
    models[3][model_num] = test_model_ev
    models[4][model_num] = score    
    
    with open('../data/models.csv', 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        for row in models:
            print("HI")
            writer.writerow(row)

    outfile.close()    

In [22]:
def print_model():
    print()
    print(test_model_name)
    print(test_model)
    print(test_model_features)
    print(test_model_ev)
    print()

In [23]:
keep_going = True
#keep_going = False
while(keep_going):
    #1. Set Features
    #get_best_features(pos_features, m, df, cur_features, labels, odds, scale=False)
    test_model_features = (get_best_features(my_pos_features, test_model, df_train, test_model_features, label_train, odds_train, 
                                             min_ev=test_model_ev))
    print_model()
    save_model()
    #2 Set hyperparameters
    #def tune_hyperparameters(input_model, input_features, input_df, input_labels, odds_input):
    test_model = tune_hyperparameters(test_model, test_model_features, df_train, label_train, odds_train, 
                                      min_ev=test_model_ev)
    print_model()
    save_model()
    #3. Set EV
    #def tune_ev(input_model, input_features, input_df, input_labels, odds_input, verbose=False):
    test_model_ev = tune_ev(test_model, test_model_features, df_train, label_train, odds_train, verbose=False)

    old_test_model_features = test_model_features #This prevents
                                                  #an uneccesary loop
    print_model()
    save_model()
    #4. Remove Features
    #def remove_to_improve(cur_features, m, df, labels, odds, scale=False, min_ev = 0):
    test_model_features = remove_to_improve(test_model_features, test_model, df_train, label_train, odds_train, min_ev = test_model_ev)    
    keep_going = False
    if old_test_model != test_model:
        print("The hyperparameters are different")
        print("OLD:")
        print(old_test_model)
        print("NEW:")
        print(test_model)
        keep_going = True
        old_test_model = test_model
    if old_test_model_features != test_model_features:
        print("The features are different")
        print("OLD:")
        print(old_test_model_features)
        print("NEW:")
        print(test_model_features)
        keep_going = True
        old_test_model_features = test_model_features
    if old_test_model_ev != test_model_ev:
        print("The EV is different")
        print("OLD:")
        print(old_test_model_ev)
        print("NEW:")
        print(test_model_ev)
        keep_going = True
        old_test_model_ev = test_model_ev
    print_model()
    save_model()
    

    

Current best score is: 25227.5260171281
NO IMPROVEMENT
FINAL BEST SCORE: 25227.5260171281

model_19
SVC(probability=True, random_state=75)
['B_total_title_bouts', 'R_win_by_TKO_Doctor_Stoppage', 'lose_streak_dif', 'R_draw', 'B_draw', 'B_Welterweight_rank', 'loss_dif', 'R_age', 'B_Stance', 'B_ev', 'R_Weight_lbs', 'B_age', 'reach_dif', 'R_avg_TD_landed', 'B_avg_TD_pct']
0



(3568, 18)

(3568,)

(3568, 2)

(248, 18)

...

(248, 18)

(248,)

(248, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-155,125,0.641779,0.358221,0
1,-195,153,0.553072,0.446928,0
2,-230,180,0.686809,0.313191,1
3,-143,113,0.426960,0.573040,0
4,-250,195,0.683069,0.316931,0
...,...,...,...,...,...
243,-230,180,0.685752,0.314248,1
244,180,-230,0.477926,0.522074,1
245,-103,-117,0.658613,0.341387,0
246,-140,120,0.428322,0.571678,0



          Number of matches: 248
          Number of bets: 205
          Number of winning bets: 88
          Number of losing bets: 118
          Number of underdog bets: 139
          Number of underdog wins: 48
          Number of underdog losses: 91
          Number of Favorite bets: 65
          Number of favorite wins: 40
          Number of favorite losses: 25
          Number of even bets: 2
          Number of even wins: 0
          Number of even losses: 2
          Profit: -1944.4560770706971
          Profit per bet: -9.485151595466816
          Profit per match: -7.840548697865715
          
          
HI
HI
HI
HI
HI


In [24]:
#Get new score:
#def evaluate_model(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels, test_odds, verbose=True):
score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

(3568, 18)

(3568,)

(3568, 2)

(248, 18)

(248,)

(248, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-155,125,0.641779,0.358221,0
1,-195,153,0.553072,0.446928,0
2,-230,180,0.686809,0.313191,1
3,-143,113,0.426960,0.573040,0
4,-250,195,0.683069,0.316931,0
...,...,...,...,...,...
243,-230,180,0.685752,0.314248,1
244,180,-230,0.477926,0.522074,1
245,-103,-117,0.658613,0.341387,0
246,-140,120,0.428322,0.571678,0



          Number of matches: 248
          Number of bets: 205
          Number of winning bets: 88
          Number of losing bets: 118
          Number of underdog bets: 139
          Number of underdog wins: 48
          Number of underdog losses: 91
          Number of Favorite bets: 65
          Number of favorite wins: 40
          Number of favorite losses: 25
          Number of even bets: 2
          Number of even wins: 0
          Number of even losses: 2
          Profit: -1944.4560770706971
          Profit per bet: -9.485151595466816
          Profit per match: -7.840548697865715
          
          


print(test_model_name) 
print(models[0][model_num])
print()
print(test_model)
print(eval(models[1][model_num]))
print()
print(test_model_features) 
print(eval(models[2][model_num]))
print()
print(test_model_ev)
print(eval(models[3][model_num]))

In [25]:
models[0][model_num] =  test_model_name 
models[1][model_num] = test_model
models[2][model_num] = test_model_features
models[3][model_num] = test_model_ev
models[4][model_num] = score

In [26]:
with open('../data/models.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    for row in models:
        print("HI")
        writer.writerow(row)
    
outfile.close()

HI
HI
HI
HI
HI


In [27]:
with open('../data/models.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    
print(len(data))

5


In [28]:
data

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_5',
  'model_6',
  'model_7',
  'model_8',
  'model_9',
  'model_10',
  'model_11',
  'model_12',
  'model_13',
  'model_14',
  'model_15',
  'model_16',
  'model_17',
  'model_18',
  'model_19'],
 ["LogisticRegression(max_iter=50000, random_state=75, solver='sag', tol=0.00012)",
  "DecisionTreeClassifier(max_depth=10, max_leaf_nodes=336,\n                       min_samples_leaf=0.0099, random_state=75,\n                       splitter='random')",
  "RandomForestClassifier(criterion='entropy', max_depth=83, max_leaf_nodes=20,\n                       min_samples_leaf=0.009999000000000001, n_estimators=140,\n                       n_jobs=-1, random_state=75)",
  'GradientBoostingClassifier(max_depth=95, min_samples_leaf=0.0015,\n                           n_estimators=195, random_state=75)',
  'GaussianNB()',
  "ExtraTreeClassifier(criterion='entropy', max_depth=83, max_leaf_nodes=20,\n                    min_sampl